Task 3

In [ ]:
"""
Manual Tool Calling Exercise (extended)
Adds a custom calculator tool (with geometry functions).
The calculator parses its input using json.loads and returns results as json.dumps.
"""

import json
import math
import ast
import operator as op
from openai import OpenAI
import ast
import math
import operator as op
import warnings

def get_weather(location: str) -> str:
    """Get the current weather for a location (simulated)."""
    weather_data = {
        "San Francisco": "Sunny, 72°F",
        "New York": "Cloudy, 55°F",
        "London": "Rainy, 48°F",
        "Tokyo": "Clear, 65°F"
    }
    return weather_data.get(location, f"Weather data not available for {location}")


import ast
import math
import operator as op

def safe_eval_expr(expr: str):
    """
    Safely evaluate a mathematical expression using ast.
    Supports numeric literals, + - * / ** unary -, parentheses,
    math.* functions/constants (sin, cos, pi, e, etc).
    This version avoids referencing ast.Num directly to prevent DeprecationWarnings.
    """
    allowed_operators = {
        ast.Add: op.add,
        ast.Sub: op.sub,
        ast.Mult: op.mul,
        ast.Div: op.truediv,
        ast.Pow: op.pow,
        ast.USub: op.neg,
        ast.UAdd: op.pos,
        ast.Mod: op.mod,
        ast.FloorDiv: op.floordiv,
    }

    allowed_names = {k: getattr(math, k) for k in dir(math) if not k.startswith("_")}
    allowed_names.setdefault("pi", math.pi)
    allowed_names.setdefault("e", math.e)

    def _eval(node):
        if isinstance(node, ast.Constant):
            if isinstance(node.value, (int, float)):
                return node.value
            raise ValueError("Unsupported constant type")
        if node.__class__.__name__ == "Num":
            val = getattr(node, "n", None)
            if isinstance(val, (int, float)):
                return val
            raise ValueError("Unsupported legacy numeric literal")

        if isinstance(node, ast.BinOp):
            op_type = type(node.op)
            if op_type not in allowed_operators:
                raise ValueError(f"Operator {op_type} not allowed")
            return allowed_operators[op_type](_eval(node.left), _eval(node.right))

        if isinstance(node, ast.UnaryOp):
            op_type = type(node.op)
            if op_type not in allowed_operators:
                raise ValueError(f"Unary operator {op_type} not allowed")
            return allowed_operators[op_type](_eval(node.operand))

        if isinstance(node, ast.Call):
            if not isinstance(node.func, ast.Name):
                raise ValueError("Only simple function calls allowed")
            func_name = node.func.id
            if func_name not in allowed_names:
                raise ValueError(f"Function {func_name} is not allowed")
            func = allowed_names[func_name]
            args = [_eval(arg) for arg in node.args]
            return func(*args)

        if isinstance(node, ast.Name):
            if node.id in allowed_names:
                return allowed_names[node.id]
            raise ValueError(f"Use of name {node.id} not allowed")

        raise ValueError(f"Unsupported AST node: {type(node)}")

    parsed = ast.parse(expr, mode='eval')
    return _eval(parsed.body)

    def _eval(node):
        if isinstance(node, ast.Constant):
            if isinstance(node.value, (int, float)):
                return node.value
            raise ValueError("Unsupported constant type")
        if isinstance(node, ast.Num): 
            return node.n

        if isinstance(node, ast.BinOp):
            op_type = type(node.op)
            if op_type not in allowed_operators:
                raise ValueError(f"Operator {op_type} not allowed")
            return allowed_operators[op_type](_eval(node.left), _eval(node.right))

        if isinstance(node, ast.UnaryOp):
            op_type = type(node.op)
            if op_type not in allowed_operators:
                raise ValueError(f"Unary operator {op_type} not allowed")
            return allowed_operators[op_type](_eval(node.operand))

        if isinstance(node, ast.Call):
            if not isinstance(node.func, ast.Name):
                raise ValueError("Only simple function calls allowed")
            func_name = node.func.id
            if func_name not in allowed_names:
                raise ValueError(f"Function {func_name} is not allowed")
            func = allowed_names[func_name]
            args = [_eval(arg) for arg in node.args]
            return func(*args)

        if isinstance(node, ast.Name):
            if node.id in allowed_names:
                return allowed_names[node.id]
            raise ValueError(f"Use of name {node.id} not allowed")

        raise ValueError(f"Unsupported AST node: {type(node)}")

    parsed = ast.parse(expr, mode='eval')
    return _eval(parsed.body)

    def _eval(node):
        if isinstance(node, ast.Num):  # <number>
            return node.n
        if isinstance(node, ast.Constant):  # Python 3.8+
            if isinstance(node.value, (int, float)):
                return node.value
            raise ValueError("Unsupported constant type")
        if isinstance(node, ast.BinOp):  # <left> <op> <right>
            op_type = type(node.op)
            if op_type not in allowed_operators:
                raise ValueError(f"Operator {op_type} not allowed")
            return allowed_operators[op_type](_eval(node.left), _eval(node.right))
        if isinstance(node, ast.UnaryOp):  # - <operand> e.g., -1
            op_type = type(node.op)
            if op_type not in allowed_operators:
                raise ValueError(f"Unary operator {op_type} not allowed")
            return allowed_operators[op_type](_eval(node.operand))
        if isinstance(node, ast.Call):  # function calls like sin(x)
            if not isinstance(node.func, ast.Name):
                raise ValueError("Only simple function calls allowed")
            func_name = node.func.id
            if func_name not in allowed_names:
                raise ValueError(f"Function {func_name} is not allowed")
            func = allowed_names[func_name]
            args = [_eval(arg) for arg in node.args]
            return func(*args)
        if isinstance(node, ast.Name):
            if node.id in allowed_names:
                return allowed_names[node.id]
            raise ValueError(f"Use of name {node.id} not allowed")
        raise ValueError(f"Unsupported AST node: {type(node)}")

    parsed = ast.parse(expr, mode='eval')
    return _eval(parsed.body)


def calculator_tool(args_json: str) -> str:
    """
    Robust calculator tool entry:
    - Accepts either a direct JSON object string e.g. '{"expr":"2+2"}'
      OR a wrapper JSON string like '{"raw":"{\"expr\":\"2+2\"}"}'
    - Always returns JSON (json.dumps).
    """
    try:
        data = json.loads(args_json)
    except Exception as e:
        return json.dumps({"error": "invalid_json_top", "message": str(e), "raw_input": args_json})

    if isinstance(data, dict) and "raw" in data:
        nested = data["raw"]
        if isinstance(nested, dict):
            data = nested
        else:
            try:
                data = json.loads(nested)
            except Exception:
                data = {"expr": nested}

    if not isinstance(data, dict):
        return json.dumps({"error": "unexpected_payload", "message": "parsed payload is not an object", "payload": str(data)})
    try:
        if "expr" in data:
            expr = str(data["expr"])
            val = safe_eval_expr(expr)
            return json.dumps({"result": val, "expression": expr})

        op_name = str(data.get("operation", "")).lower()
        if op_name in ("add", "sum"):
            a = float(data["a"]); b = float(data["b"])
            return json.dumps({"result": a + b})
        if op_name in ("sub", "subtract", "minus"):
            a = float(data["a"]); b = float(data["b"])
            return json.dumps({"result": a - b})
        if op_name in ("mul", "multiply"):
            a = float(data["a"]); b = float(data["b"])
            return json.dumps({"result": a * b})
        if op_name in ("div", "divide"):
            a = float(data["a"]); b = float(data["b"])
            if b == 0:
                return json.dumps({"error": "division_by_zero"})
            return json.dumps({"result": a / b})
        if op_name in ("pow", "power"):
            a = float(data["a"]); b = float(data["b"])
            return json.dumps({"result": a ** b})

        if op_name in ("sin", "cos", "tan", "sqrt", "log", "ln", "exp"):
            x_expr = data.get("x")
            if x_expr is None:
                return json.dumps({"error": "missing_argument", "message": "x required"})
            x_val = safe_eval_expr(str(x_expr))
            if op_name == "sin":
                return json.dumps({"result": math.sin(x_val)})
            if op_name == "cos":
                return json.dumps({"result": math.cos(x_val)})
            if op_name == "tan":
                return json.dumps({"result": math.tan(x_val)})
            if op_name == "sqrt":
                return json.dumps({"result": math.sqrt(x_val)})
            if op_name in ("log", "ln"):
                return json.dumps({"result": math.log(x_val)})
            if op_name == "exp":
                return json.dumps({"result": math.exp(x_val)})

        if op_name == "area_circle":
            r = float(data["radius"])
            return json.dumps({"result": math.pi * r * r})
        if op_name == "circumference":
            r = float(data["radius"])
            return json.dumps({"result": 2 * math.pi * r})
        if op_name == "area_rectangle":
            w = float(data["width"]); h = float(data["height"])
            return json.dumps({"result": w * h})
        if op_name == "perimeter_rectangle":
            w = float(data["width"]); h = float(data["height"])
            return json.dumps({"result": 2 * (w + h)})
        if op_name == "area_triangle":
            b = float(data["base"]); h = float(data["height"])
            return json.dumps({"result": 0.5 * b * h})

        return json.dumps({"error": "unknown_operation", "message": "operation not recognized", "input": data})
    except Exception as e:
        return json.dumps({"error": "execution_error", "message": str(e), "input": data})


tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get the current weather for a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {"type": "string", "description": "The city name, e.g. San Francisco"}
                },
                "required": ["location"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "calculator",
            "description": (
                "Perform numeric calculations and geometry helpers. "
                "Input must be a JSON string. Examples: "
                '{"expr":"2+2"}, '
                '{"operation":"area_circle","radius":3}, '
                '{"operation":"sin","x":"pi/2"}'
            ),
            "parameters": {
                "type": "object",
                "properties": {
                    "raw": {"type": "string", "description": "A JSON string that the calculator tool will parse"}
                },
                "required": ["raw"]
            }
        }
    }
]

SYSTEM_PROMPT = (
    "You are a helpful assistant. You have two tools: a weather tool and a calculator tool.\n\n"
    "CRITICAL: For ANY mathematical operation (even simple arithmetic), you MUST call the calculator tool. "
    "Do not attempt to compute numeric results yourself. The calculator expects a JSON string argument. "
    "E.g., call calculator with {'raw': '{\"expr\":\"2+2\"}'} or {'raw':'{\"operation\":\"area_circle\",\"radius\":3}'}.\n\n"
    "For weather requests, call the weather tool. After receiving tool output, present the result to the user."
)

def run_agent(user_query: str):
    """
    Simple agent that can use tools. Shows a manual tool-call loop.
    """
    client = OpenAI()  

    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_query}
    ]

    print(f"User: {user_query}\n")

    for iteration in range(5):
        print(f"--- Iteration {iteration + 1} ---")
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            tools=tools,
            tool_choice="auto"
        )

        assistant_message = response.choices[0].message
        if getattr(assistant_message, "tool_calls", None):
            print(f"LLM wants to call {len(assistant_message.tool_calls)} tool(s)")
            messages.append(assistant_message)
            for tool_call in assistant_message.tool_calls:
                function_name = tool_call.function.name
                raw_args_json = tool_call.function.arguments

                print(f"  Tool: {function_name}")
                print(f"  Raw Args (string): {raw_args_json}")

                if function_name == "get_weather":
                    try:
                        parsed_args = json.loads(raw_args_json)
                    except Exception as e:
                        result = f"Error: invalid JSON for weather args: {e}"
                    else:
                        location = parsed_args.get("location")
                        result = get_weather(location)
                elif function_name == "calculator":
                    result = calculator_tool(raw_args_json)
                else:
                    result = f"Error: Unknown function {function_name}"

                print(f"  Result: {result}")
                messages.append({
                    "role": "tool",
                    "tool_call_id": getattr(tool_call, "id", None),
                    "name": function_name,
                    "content": result
                })

            print()
        else:
            final_text = getattr(assistant_message, "content", None)
            print(f"Assistant: {final_text}\n")
            return final_text

    return "Max iterations reached"


if __name__ == "__main__":
    examples = [
        "What is 2 + 2?",
        "Compute the area of a circle with radius 3",
        "What's sin(pi/2) plus 5?",
        "What's the weather in San Francisco?",
        "Calculate the area of triangle with base 10 and height 6"
    ]
    system_prompt = """You are a helpful assistant with access to a calculator tool.
                  CRITICAL: You must ALWAYS use the calculator tool for ANY mathematical operations, including:
                  - Basic arithmetic (addition, subtraction, multiplication, division)
                  - Comparisons and numerical evaluations
                  - Any calculation involving numbers

                  Never attempt to calculate results yourself. Always call the calculator tool, even for simple operations like 2+2.

                  After receiving the tool result, present it to the user naturally."""

    for i,ex in enumerate(examples):
        print(f"Test {i+1}:\n")
        run_agent(system_prompt + " " +ex)
        print("\n")

Test 1:

User: You are a helpful assistant with access to a calculator tool.
                  CRITICAL: You must ALWAYS use the calculator tool for ANY mathematical operations, including:
                  - Basic arithmetic (addition, subtraction, multiplication, division)
                  - Comparisons and numerical evaluations
                  - Any calculation involving numbers

                  Never attempt to calculate results yourself. Always call the calculator tool, even for simple operations like 2+2.

                  After receiving the tool result, present it to the user naturally. What is 2 + 2?

--- Iteration 1 ---
LLM wants to call 1 tool(s)
  Tool: calculator
  Raw Args (string): {"raw":"{\"expr\":\"2+2\"}"}
  Result: {"result": 4, "expression": "2+2"}

--- Iteration 2 ---
Assistant: The result of 2 + 2 is 4.



Test 2:

User: You are a helpful assistant with access to a calculator tool.
                  CRITICAL: You must ALWAYS use the calculator tool for ANY 